In [27]:
!pip install datasets


DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/dnspython-2.1.1.dev472+g8231eaa-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [28]:
import pandas as pd
from datasets import load_dataset

# Load dataset
dataset = load_dataset("Deysi/sentences-and-emotions")

# Convert to pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Check the data
df.head()


,utterance,emotion
0,The blake's got divorced .,neutral
1,Really ? Why ?,neutral
2,Mr . black has been getting a little around as...,neutral
3,I'm surprised . He does't look like a guy who'...,surprise
4,"No , he doesn't . But his wife found out he ha...",neutral


In [29]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

# Tokenize and preprocess sentences
sentences = df['utterance'].apply(simple_preprocess).tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)


In [30]:
import numpy as np

def sentence_to_vector(sentence, model):
    words = simple_preprocess(sentence)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Convert all sentences to vectors
X = np.array([sentence_to_vector(sentence, word2vec_model) for sentence in df['utterance']])
y = df['emotion']


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize AdaBoost Classifier with Decision Tree as the base estimator
ada_classifier = AdaBoostClassifier(n_estimators=100, random_state=42)

# Train the model
ada_classifier.fit(X_train, y_train)


/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(n_estimators=100, random_state=42)

In [32]:
# Make predictions
y_pred = ada_classifier.predict(X_test)
# Convert the new utterance to a feature vector
def convert_to_vector(sentence, model):
    return sentence_to_vector(sentence, model)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Example usage
new_utterance = "I feel happy today"
new_utterance_vector = convert_to_vector(new_utterance, word2vec_model)
predicted_emotion = ada_classifier.predict([new_utterance_vector])
print(f"Predicted emotion: {predicted_emotion[0]}")

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))



Accuracy: 0.5655
Predicted emotion: neutral
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        24
     disgust       0.00      0.00      0.00         3
        fear       0.00      0.00      0.00         6
   happiness       0.00      0.00      0.00       134
     neutral       0.57      1.00      0.72       272
     sadness       0.00      0.00      0.00        26
    surprise       0.00      0.00      0.00        16

    accuracy                           0.57       481
   macro avg       0.08      0.14      0.10       481
weighted avg       0.32      0.57      0.41       481



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
!pip install googletrans==4.0.0-rc1

import pandas as pd
import numpy as np
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from googletrans import Translator

# Download necessary NLTK data
nltk.download('punkt')

# Load the dataset
dataset = load_dataset("Deysi/sentences-and-emotions")
df = pd.DataFrame(dataset['train'])

# Initialize the translator
translator = Translator()

def back_translate(text, src_lang='en', tgt_lang='fr'):
    try:
        # Translate from source to target language
        translated = translator.translate(text, src=src_lang, dest=tgt_lang).text
        # Translate back to source language
        back_translated = translator.translate(translated, src=tgt_lang, dest=src_lang).text
        return back_translated
    except:
        return text  # Return original text if translation fails

# Step 1: Identify the largest class size
class_counts = df['emotion'].value_counts()
max_class_size = class_counts.max()

# Step 2: Augment the smaller classes
augmented_texts = []
augmented_labels = []
for label, count in class_counts.items():
    if count < max_class_size:
        # Filter texts for this label
        subset = df[df['emotion'] == label]
        # Calculate how many samples are needed to reach max_class_size
        num_to_augment = max_class_size - count
        # Generate augmented texts
        augmented_samples = subset['utterance'].apply(lambda x: back_translate(x))
        # Ensure we augment enough samples
        augmented_samples = list(augmented_samples)
        while len(augmented_samples) < num_to_augment:
            augmented_samples.extend(augmented_samples)  # Duplicate to ensure we have enough
        # Sample to the required number
        sampled_augmented = np.random.choice(augmented_samples, num_to_augment, replace=False)
        augmented_texts.extend(sampled_augmented)
        augmented_labels.extend([label] * num_to_augment)

# Create augmented DataFrame
augmented_df = pd.DataFrame({'utterance': augmented_texts, 'emotion': augmented_labels})

# Combine the original and augmented datasets
balanced_df = pd.concat([df, augmented_df]).reset_index(drop=True)

print("Class distribution after balancing:")
print(balanced_df['emotion'].value_counts())

# Preprocess the text
def preprocess(text):
    return word_tokenize(text.lower())

balanced_df['tokens'] = balanced_df['utterance'].apply(preprocess)

# Create Word2Vec model
w2v_model = Word2Vec(sentences=balanced_df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# Create sentence embeddings by averaging word vectors
def get_sentence_embedding(tokens):
    return np.mean([w2v_model.wv[word] for word in tokens if word in w2v_model.wv], axis=0)

balanced_df['embedding'] = balanced_df['tokens'].apply(get_sentence_embedding)

# Prepare features and labels
X = np.array(balanced_df['embedding'].tolist())
y = balanced_df['emotion']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train AdaBoost classifier
adaboost = AdaBoostClassifier(n_estimators=100, random_state=42)
adaboost.fit(X_train, y_train)

# Make predictions
y_pred = adaboost.predict(X_test)

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/dnspython-2.1.1.dev472+g8231eaa-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.0 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17398

[nltk_data] Downloading package punkt to /Users/ashwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Class distribution after balancing:
emotion
neutral      1306
surprise     1306
anger        1306
disgust      1306
happiness    1306
sadness      1306
excited      1306
sad          1306
fear         1306
happines     1306
Name: count, dtype: int64


/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       267
     disgust       0.00      0.00      0.00       266
     excited       1.00      1.00      1.00       282
        fear       0.00      0.00      0.00       270
    happines       0.00      0.00      0.00       258
   happiness       0.00      0.00      0.00       288
     neutral       0.00      0.00      0.00       252
         sad       1.00      1.00      1.00       236
     sadness       0.12      1.00      0.21       241
    surprise       0.00      0.00      0.00       252

    accuracy                           0.29      2612
   macro avg       0.21      0.30      0.22      2612
weighted avg       0.21      0.29      0.22      2612



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
import numpy as np

# Load dataset
data = pd.read_csv('sampled_data.csv', header=None, names=['text', 'label'])

# Limit the dataset to 400 samples per label
sampled_data = data.groupby('label').apply(lambda x: x.sample(n=400, random_state=42)).reset_index(drop=True)

# Encode labels
le = LabelEncoder()
sampled_data['label'] = le.fit_transform(sampled_data['label'])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(sampled_data['text'], sampled_data['label'], test_size=0.2, random_state=42)


ValueError: Cannot take a larger sample than population when 'replace=False'

In [ ]:
from gensim.utils import simple_preprocess

# Tokenize the text
tokenized_sentences = [simple_preprocess(sentence) for sentence in sampled_data['text']]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)


In [ ]:
def sentence_to_vector(sentence, model):
    tokens = simple_preprocess(sentence)
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# Transform sentences to vectors
X_train_vec = np.array([sentence_to_vector(sentence, w2v_model) for sentence in X_train])
X_test_vec = np.array([sentence_to_vector(sentence, w2v_model) for sentence in X_test])


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Initialize AdaBoost with a Decision Tree classifier
ada_model = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=50, random_state=42)

# Train the model
ada_model.fit(X_train_vec, y_train)

# Evaluate the model
y_pred = ada_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
# Predict a new sentence
def predict_sentence(sentence, model, w2v_model):
    vector = sentence_to_vector(sentence, w2v_model)
    prediction = model.predict([vector])
    return le.inverse_transform(prediction)[0]

# Test prediction
new_sentence = "I'm frustrated with not getting into the school I wanted."
print(f'Prediction: {predict_sentence(new_sentence, ada_model, w2v_model)}')


In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv('sampled_data.csv', header=None, names=['text', 'label'], delimiter=',', quoting=3)

# Split data into features and labels
X = df['text']
y = df['label']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Convert text data to numerical data using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Train an AdaBoost classifier with a DecisionTree as base estimator
model = AdaBoostClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vect, y_train)

# Predict on the test set
y_pred = model.predict(X_test_vect)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Test the model with new examples
new_examples = [
    "I am so excited about the upcoming vacation!",
    "I feel completely overwhelmed with work right now.",
    "The kids are really making me angry today."
]

# Convert new examples to numerical data using the same vectorizer
new_examples_vect = vectorizer.transform(new_examples)

# Predict emotions for the new examples
predictions = model.predict(new_examples_vect)

# Output predictions
for example, prediction in zip(new_examples, predictions):
    print(f"Text: {example}")
    print(f"Predicted Emotion: {prediction}")
    print()


/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Accuracy: 0.2644444444444444
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       162
        fear       0.31      0.13      0.18       151
         joy       0.00      0.00      0.00       148
        love       0.00      0.00      0.00       143
     sadness       0.19      0.98      0.32       141
    surprise       0.91      0.52      0.66       155

    accuracy                           0.26       900
   macro avg       0.23      0.27      0.19       900
weighted avg       0.24      0.26      0.19       900

Text: I am so excited about the upcoming vacation!
Predicted Emotion: sadness

Text: I feel completely overwhelmed with work right now.
Predicted Emotion: fear

Text: The kids are really making me angry today.
Predicted Emotion: sadness



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
